Import Libraries

In [1]:
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
from scipy import interpolate

# Import pandas
import pandas as pd

# Import numpy
import numpy as np
from numpy import * 
from numpy.linalg import multi_dot
import numpy_financial as npf

# Plot settings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = 16, 8

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# IBT or Pension or Retirement
PLAN = 'IBT'

UPS_Contr_Pctg = 0

Years_to_FF = 20


In [2]:
df_CF = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "PBO Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.5,1.204791e+09,2.626341e+08,4.239987e+08
2022-12-31,1.5,1.261684e+09,2.935988e+08,4.570359e+08
2023-12-31,2.5,1.322496e+09,3.253961e+08,4.883641e+08
2024-12-31,3.5,1.382473e+09,3.576365e+08,5.168109e+08
2025-12-31,4.5,1.441835e+09,3.898680e+08,5.408954e+08
...,...,...,...,...
2096-12-31,75.5,1.769334e+05,3.948190e+06,1.620103e+06
2097-12-31,76.5,1.503230e+05,2.959047e+06,1.183430e+06
2098-12-31,77.5,1.300395e+05,2.180928e+06,8.499443e+05


In [3]:
df_CF = df_CF.divide(12)
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-12-31,0.125000,1.051403e+08,2.446657e+07,3.808632e+07
2023-12-31,0.208333,1.102080e+08,2.711634e+07,4.069701e+07
2024-12-31,0.291667,1.152061e+08,2.980304e+07,4.306757e+07
2025-12-31,0.375000,1.201529e+08,3.248900e+07,4.507462e+07
...,...,...,...,...
2096-12-31,6.291667,1.474445e+04,3.290158e+05,1.350085e+05
2097-12-31,6.375000,1.252692e+04,2.465872e+05,9.861913e+04
2098-12-31,6.458333,1.083662e+04,1.817440e+05,7.082869e+04


In [4]:
df=df_CF.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_CF = df
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-02-28,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-03-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-04-30,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [5]:
df_CF_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "CF", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,19389650.5,31075638.75
2021-11-30,0.166667,9.387313e+07,19389650.5,31075638.75
2021-12-31,0.250000,9.387313e+07,19389650.5,31075638.75


In [6]:
df_CF  = df_CF_add.append(df_CF)
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [7]:
from itertools import count, takewhile
def frange(start, stop, step):
        return takewhile(lambda x: x< stop, count(start, step))
t = list(frange((1/12), 79.41666666666, (1/12)))

In [8]:
df_t=pd.DataFrame(t)
df_t

,0
0,0.083333
1,0.166667
2,0.250000
3,0.333333
4,0.416667
...,...
947,79.000000
948,79.083333
949,79.166667
950,79.250000


In [9]:
df_CF['Time'] = df_t.values
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [10]:
#IBT = df_CF[PLAN]
#RP = df_CF[PLAN]
CF = np.array(df_CF[PLAN])
Time_CF = df_CF.index
DF = np.array(df_CF['Time'])
CF

array([3.10756388e+07, 3.10756388e+07, 3.10756388e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.53332242e+07,
       3.53332242e+07, 3.53332242e+07, 3.53332242e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 3.80863224e+07,
       3.80863224e+07, 3.80863224e+07, 3.80863224e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.06970095e+07,
       4.06970095e+07, 4.06970095e+07, 4.06970095e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.30675742e+07,
       4.30675742e+07, 4.30675742e+07, 4.30675742e+07, 4.50746205e+07,
       4.50746205e+07, 4.50746205e+07, 4.50746205e+07, 4.50746205e+07,
      

In [11]:
#from PyCurve.curve import Curve

In [12]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 145)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,2010-09-30 00:00:00,2010-08-31 00:00:00,2010-07-31 00:00:00,2010-06-30 00:00:00,2010-05-31 00:00:00,2010-04-30 00:00:00,2010-03-31 00:00:00,2010-02-28 00:00:00,2010-01-31 00:00:00,2009-12-31 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,0.896931,0.980827,1.039656,1.336495,1.356106,0.796529,0.824371,0.959171,0.854822,0.997499
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,1.012479,1.093406,1.156291,1.494602,1.548488,1.069795,1.179135,1.183362,1.099006,1.386799
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,1.065754,1.180900,1.244818,1.590332,1.686592,1.323298,1.459969,1.412837,1.358586,1.664896
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,1.132115,1.250318,1.346470,1.693166,1.853045,1.533786,1.684716,1.612329,1.567095,1.892898
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,1.289564,1.400126,1.517540,1.883170,2.099087,1.825434,1.982464,1.898602,1.860517,2.202034
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,1.480375,1.593382,1.723042,2.141054,2.396014,2.163991,2.328185,2.229761,2.204905,2.555180
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,1.695568,1.812045,1.960053,2.417479,2.679186,2.485899,2.676519,2.544241,2.541772,2.898791
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,1.924112,2.035995,2.214525,2.671984,2.922165,2.770801,2.999055,2.823711,2.849782,3.213513
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,2.149881,2.249067,2.462229,2.882171,3.138009,3.038841,3.281442,3.099499,3.139972,3.515488
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,2.374888,2.459324,2.706179,3.066176,3.344929,3.302149,3.534706,3.380740,3.421418,3.802621


In [13]:
raw_liab_ibt_dict={}
tr = Raw_FTSE_YC['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict[col])
#curve = Curve(t,y)
#print("curve.get_rate:{},curve.get_time:{}".format(curve.get_rate,curve.get_time))
#print(curve.get_rate)
#print(curve.get_time)

952

In [14]:
int_df = pd.DataFrame(raw_liab_ibt_dict)
int_df = int_df.drop(['Date'], axis=1)
int_df = int_df.iloc[:, ::-1]
int_df

,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
1,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
2,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
3,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
4,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
948,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
949,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
950,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599


In [15]:
liab_plan_dict={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df[col][j]/100)**DF[j]))
    liab_plan_dict[col] = temp_pv
len(liab_plan_dict)

142

In [16]:
pv_df = pd.DataFrame(liab_plan_dict, index = ['Present Values']).transpose()
np.negative(pv_df['Present Values'])
pv_df

,Present Values
2009-12-31,9.441978e+09
2010-01-31,9.531003e+09
2010-02-28,9.462231e+09
2010-03-31,9.378239e+09
2010-04-30,9.723118e+09
...,...
2021-05-31,1.492173e+10
2021-06-30,1.556223e+10
2021-07-31,1.594887e+10
2021-08-31,1.580312e+10


In [17]:
Plan_Return_YC = np.zeros(len(liab_plan_dict)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC[i] = ((pv_df['Present Values'][i+1])/pv_df['Present Values'][i])-1
Plan_Return_YC

array([ 9.42865717e-03, -7.21562554e-03, -8.87648229e-03,  3.67743780e-02,
       -4.95219251e-03,  5.12721060e-02,  1.06388756e-02,  5.75074062e-02,
       -1.38859807e-02, -3.79158140e-02, -6.56062121e-03, -1.63883857e-02,
       -3.11037341e-02,  1.69336215e-02, -1.38167844e-02,  2.30058404e-02,
        1.85198215e-02, -2.78849551e-02,  4.63135011e-02,  2.31859042e-02,
        7.33892885e-02, -4.45204518e-04,  6.06054040e-04,  4.55745101e-02,
       -4.07416611e-03,  1.94650863e-03, -3.51593719e-02,  2.13459578e-02,
        3.16994260e-02,  3.45324164e-02,  6.68892757e-02, -1.48678257e-02,
       -1.70845231e-02,  1.86894075e-02, -1.27808231e-02, -2.24026719e-02,
       -4.00684167e-02,  7.69536834e-03, -9.30247813e-03,  4.08419403e-02,
       -6.73129263e-02, -4.76998550e-02, -1.49139088e-03, -7.42057291e-03,
       -7.00084844e-03,  1.70391373e-02, -1.72789513e-02, -5.30234015e-03,
        4.17969220e-02,  1.43274467e-02,  5.67373578e-03,  1.67103052e-02,
        1.90175386e-02, -

In [18]:
Plan_Returns_YC = pd.DataFrame(Plan_Return_YC)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC.to_excel(writer, sheet_name='returns')
pv_df.to_excel(writer, sheet_name='pv')
writer.save()

In [19]:
df_SC = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "Service Cost Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC

,Retirement,Pension,IBT
Year,,,
2021-12-31,1511573.39,1.515507e+05,8.562248e+05
2022-12-31,4341798.12,6.378839e+05,3.265973e+06
2023-12-31,6325185.11,1.364457e+06,6.377871e+06
2024-12-31,8362114.74,2.184367e+06,9.520245e+06
2025-12-31,10514285.02,3.091714e+06,1.271774e+07
...,...,...,...
2096-12-31,2534.24,7.448368e+05,8.144005e+05
2097-12-31,1502.48,5.765053e+05,6.166164e+05
2098-12-31,873.21,4.384455e+05,4.589457e+05


In [20]:
df_SC = df_SC.divide(12)
df_SC

,Retirement,Pension,IBT
Year,,,
2021-12-31,125964.449167,12629.223266,7.135207e+04
2022-12-31,361816.510000,53156.995293,2.721644e+05
2023-12-31,527098.759167,113704.742421,5.314892e+05
2024-12-31,696842.895000,182030.602914,7.933538e+05
2025-12-31,876190.418333,257642.865694,1.059812e+06
...,...,...,...
2096-12-31,211.186667,62069.734330,6.786670e+04
2097-12-31,125.206667,48042.110266,5.138470e+04
2098-12-31,72.767500,36537.126910,3.824548e+04


In [21]:
df=df_SC.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_SC = df
df_SC

,Retirement,Pension,IBT
date,,,
2021-12-31,125964.449167,12629.223266,71352.069903
2022-01-31,125964.449167,12629.223266,71352.069903
2022-02-28,125964.449167,12629.223266,71352.069903
2022-03-31,125964.449167,12629.223266,71352.069903
2022-04-30,125964.449167,12629.223266,71352.069903
...,...,...,...
2100-08-31,41.463333,27272.299072,27955.270099
2100-09-30,41.463333,27272.299072,27955.270099
2100-10-31,41.463333,27272.299072,27955.270099


In [22]:
df_SC_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "SC", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333


In [23]:
df_SC  = df_SC_add.append(df_SC)
df_SC

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333
2021-12-31,NaN,125964.449167,12629.223266,71352.069903
2022-01-31,NaN,125964.449167,12629.223266,71352.069903
...,...,...,...,...
2100-08-31,NaN,41.463333,27272.299072,27955.270099
2100-09-30,NaN,41.463333,27272.299072,27955.270099
2100-10-31,NaN,41.463333,27272.299072,27955.270099


In [24]:
col_SC = df_SC.columns
col_CF = df_CF.columns
df_CF_Tot = UPS_Contr_Pctg*df_SC[col_SC] + df_CF[col_CF]
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04


In [25]:
df_CF_Tot['Time'] = df_t.values
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [26]:
CF_Tot = df_CF_Tot[PLAN]
Time_CF_Tot = df_CF_Tot.index
DF_Tot=df_CF_Tot['Time']

In [27]:
liab_plan_dict_tot={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot[j]/((1+int_df[col][j]/100)**DF_Tot[j]))
    liab_plan_dict_tot[col] = temp_pv
liab_plan_dict_tot


{datetime.datetime(2009, 12, 31, 0, 0): 9441977545.491364,
 datetime.datetime(2010, 1, 31, 0, 0): 9531002714.756456,
 datetime.datetime(2010, 2, 28, 0, 0): 9462230568.192614,
 datetime.datetime(2010, 3, 31, 0, 0): 9378239246.162477,
 datetime.datetime(2010, 4, 30, 0, 0): 9723118160.902142,
 datetime.datetime(2010, 5, 31, 0, 0): 9674967407.949291,
 datetime.datetime(2010, 6, 30, 0, 0): 10171023362.43103,
 datetime.datetime(2010, 7, 31, 0, 0): 10279231614.6664,
 datetime.datetime(2010, 8, 31, 0, 0): 10870363562.641134,
 datetime.datetime(2010, 9, 30, 0, 0): 10719417903.831335,
 datetime.datetime(2010, 10, 31, 0, 0): 10312982448.670992,
 datetime.datetime(2010, 11, 30, 0, 0): 10245322877.249722,
 datetime.datetime(2010, 12, 31, 0, 0): 10077418574.04642,
 datetime.datetime(2011, 1, 31, 0, 0): 9763973226.147701,
 datetime.datetime(2011, 2, 28, 0, 0): 9929312652.788202,
 datetime.datetime(2011, 3, 31, 0, 0): 9792121480.412828,
 datetime.datetime(2011, 4, 30, 0, 0): 10017397464.769346,
 datet

In [28]:
pv_df_tot = pd.DataFrame(liab_plan_dict_tot, index = ['Present Values']).transpose()

pv_df_tot

,Present Values
2009-12-31,9.441978e+09
2010-01-31,9.531003e+09
2010-02-28,9.462231e+09
2010-03-31,9.378239e+09
2010-04-30,9.723118e+09
...,...
2021-05-31,1.492173e+10
2021-06-30,1.556223e+10
2021-07-31,1.594887e+10
2021-08-31,1.580312e+10


In [29]:
Plan_Return_YC_Tot = np.zeros(len(liab_plan_dict_tot)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Tot[i] = ((pv_df_tot['Present Values'][i+1])/pv_df_tot['Present Values'][i])-1
Plan_Returns_YC_Tot = pd.DataFrame(Plan_Return_YC_Tot)
Plan_Returns_YC_Tot

,0
0,0.009429
1,-0.007216
2,-0.008876
3,0.036774
4,-0.004952
...,...
136,0.002709
137,0.042924
138,0.024844
139,-0.009138


In [30]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability & SC Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Tot.to_excel(writer, sheet_name='returns')
pv_df_tot.to_excel(writer, sheet_name='pv')
writer.save()

Calculate IRR's

In [31]:
Plan_DR = pd.read_excel("UPS Pension - Historical Liability Data - 9.30.21.xlsx",sheet_name = PLAN ,skiprows = [0,1,2,3],usecols=[1,2], na_values=[""],index_col=0)
#dataset.set_index('Year')
Plan_DR


,IRR
Date,
2009-12-31,0.059959
2010-01-31,0.058975
2010-02-28,0.059820
2010-03-31,0.060101
2010-04-30,0.057504
...,...
2021-05-31,0.030762
2021-06-30,0.028554
2021-07-31,0.027088


In [32]:
#col_DR = Plan_DR.columns
#df_DR_M =  (1+Plan_DR)**(1/12)-1
#df_DR_M = Plan_DR
IRR = Plan_DR['IRR']
IRR

Date
2009-12-31    0.059959
2010-01-31    0.058975
2010-02-28    0.059820
2010-03-31    0.060101
2010-04-30    0.057504
                ...   
2021-05-31    0.030762
2021-06-30    0.028554
2021-07-31    0.027088
2021-08-31    0.027607
2021-09-30    0.032400
Name: IRR, Length: 142, dtype: float64

In [33]:
#liab_plan_dict_dr={}
#for col in Plan_DR.columns:
    #print('col: {}'.format(col))
#temp_pv=0
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
    #for j in range (0,len(Time_CF)):
        temp_pv += (CF[i]/((1+Plan_DR['IRR'][j]**DF[i])))
    #liab_plan_dict_dr[col] = temp_pv
    PV_DR = temp_pv
PV_DR

3974884888714.809

In [34]:
temp_pv = (CF[0]/((1+Plan_DR['IRR'][-1])**DF[0]))
temp_pv

30993174.718990065

In [35]:
Plan_PV_DR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
        Plan_PV_DR[j] += (CF[i]/((1+Plan_DR['IRR'][j])**DF[i]))
Plan_PV_DR

array([9.29336565e+09, 9.42045483e+09, 9.31104495e+09, 9.27523915e+09,
       9.61590533e+09, 9.46050351e+09, 9.93155311e+09, 1.00908218e+10,
       1.06079164e+10, 1.05083114e+10, 1.01387245e+10, 1.00525102e+10,
       9.96091844e+09, 9.65217195e+09, 9.84778344e+09, 9.70762421e+09,
       9.94852305e+09, 1.01100236e+10, 9.83579907e+09, 1.01947065e+10,
       1.03539345e+10, 1.09062177e+10, 1.09699870e+10, 1.10204328e+10,
       1.15061017e+10, 1.14824835e+10, 1.15329407e+10, 1.12114178e+10,
       1.14341247e+10, 1.17581639e+10, 1.22349737e+10, 1.30105628e+10,
       1.28834870e+10, 1.27119687e+10, 1.29644103e+10, 1.28048406e+10,
       1.25407661e+10, 1.20454800e+10, 1.21517371e+10, 1.20283597e+10,
       1.25086580e+10, 1.16919306e+10, 1.11144060e+10, 1.11325889e+10,
       1.10718694e+10, 1.10035188e+10, 1.12151338e+10, 1.10278015e+10,
       1.09533609e+10, 1.14312761e+10, 1.15971597e+10, 1.16675468e+10,
       1.18574817e+10, 1.20951074e+10, 1.20765841e+10, 1.20680286e+10,
      

In [36]:
pv_df_dr = pd.DataFrame(Plan_PV_DR, columns=['Present Values'])
#np.negative(pv_df_dr['Present Values'])
test = np.append(np.negative(pv_df_dr['Present Values'][0]),CF)
#pv_df_dr['Present Values']
test

array([-9.29336565e+09,  3.10756388e+07,  3.10756388e+07,  3.10756388e+07,
        3.53332242e+07,  3.53332242e+07,  3.53332242e+07,  3.53332242e+07,
        3.53332242e+07,  3.53332242e+07,  3.53332242e+07,  3.53332242e+07,
        3.53332242e+07,  3.53332242e+07,  3.53332242e+07,  3.53332242e+07,
        3.80863224e+07,  3.80863224e+07,  3.80863224e+07,  3.80863224e+07,
        3.80863224e+07,  3.80863224e+07,  3.80863224e+07,  3.80863224e+07,
        3.80863224e+07,  3.80863224e+07,  3.80863224e+07,  3.80863224e+07,
        4.06970095e+07,  4.06970095e+07,  4.06970095e+07,  4.06970095e+07,
        4.06970095e+07,  4.06970095e+07,  4.06970095e+07,  4.06970095e+07,
        4.06970095e+07,  4.06970095e+07,  4.06970095e+07,  4.06970095e+07,
        4.30675742e+07,  4.30675742e+07,  4.30675742e+07,  4.30675742e+07,
        4.30675742e+07,  4.30675742e+07,  4.30675742e+07,  4.30675742e+07,
        4.30675742e+07,  4.30675742e+07,  4.30675742e+07,  4.30675742e+07,
        4.50746205e+07,  

In [37]:
#pv_df_dr = pd.DataFrame(Plan_PV_DR)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability PV Using DR.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
pv_df_dr.to_excel(writer, sheet_name='Present Values')
writer.save()

In [38]:
import numpy as np
from scipy.optimize import fsolve

def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

def irr(cfs, yrs, x0, **kwargs):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs), **kwargs))

In [39]:
IRR = np.zeros(len(pv_df))
for j in range (len(pv_df)):
    cash_flow = np.append(np.negative(pv_df['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.05881071, 0.05813718, 0.05865657, 0.05929937, 0.05671823,
       0.05706954, 0.05358092, 0.05285641, 0.04910549, 0.0500316 ,
       0.05263295, 0.05308212, 0.0542178 , 0.05642237, 0.05524537,
       0.05621971, 0.05463125, 0.05336608, 0.05532199, 0.05220733,
       0.05066326, 0.04602532, 0.04605388, 0.04601503, 0.04319474,
       0.04345007, 0.04332837, 0.0455904 , 0.04424987, 0.04229903,
       0.04021601, 0.03635177, 0.03723348, 0.03825684, 0.03715757,
       0.03792001, 0.03927629, 0.04176828, 0.04129672, 0.04187191,
       0.03942961, 0.04371754, 0.04682956, 0.046926  , 0.04740859,
       0.04786575, 0.04676954, 0.0479006 , 0.04824794, 0.04560087,
       0.04469608, 0.04433832, 0.04329714, 0.04212557, 0.04230866,
       0.04230614, 0.04015171, 0.04223122, 0.04098365, 0.03991162,
       0.03869576, 0.03411321, 0.036815  , 0.0367148 , 0.03862506,
       0.04046132, 0.04332333, 0.04202402, 0.04279736, 0.04216219,
       0.04200598, 0.04211836, 0.04239294, 0.04184409, 0.04044

In [40]:
IRR = np.zeros(len(pv_df_tot))
for j in range (len(pv_df_tot)):
    cash_flow = np.append(np.negative(pv_df_tot['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.05881071, 0.05813718, 0.05865657, 0.05929937, 0.05671823,
       0.05706954, 0.05358092, 0.05285641, 0.04910549, 0.0500316 ,
       0.05263295, 0.05308212, 0.0542178 , 0.05642237, 0.05524537,
       0.05621971, 0.05463125, 0.05336608, 0.05532199, 0.05220733,
       0.05066326, 0.04602532, 0.04605388, 0.04601503, 0.04319474,
       0.04345007, 0.04332837, 0.0455904 , 0.04424987, 0.04229903,
       0.04021601, 0.03635177, 0.03723348, 0.03825684, 0.03715757,
       0.03792001, 0.03927629, 0.04176828, 0.04129672, 0.04187191,
       0.03942961, 0.04371754, 0.04682956, 0.046926  , 0.04740859,
       0.04786575, 0.04676954, 0.0479006 , 0.04824794, 0.04560087,
       0.04469608, 0.04433832, 0.04329714, 0.04212557, 0.04230866,
       0.04230614, 0.04015171, 0.04223122, 0.04098365, 0.03991162,
       0.03869576, 0.03411321, 0.036815  , 0.0367148 , 0.03862506,
       0.04046132, 0.04332333, 0.04202402, 0.04279736, 0.04216219,
       0.04200598, 0.04211836, 0.04239294, 0.04184409, 0.04044

In [41]:
IRR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    cash_flow = np.append(np.negative(pv_df_dr['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.0599586 , 0.05897511, 0.05982047, 0.06010068, 0.05750438,
       0.0586697 , 0.05522964, 0.05412602, 0.05072926, 0.05136254,
       0.0537996 , 0.0543889 , 0.055024  , 0.05723685, 0.05582173,
       0.05683097, 0.05511068, 0.05399487, 0.0559071 , 0.05342129,
       0.0523634 , 0.04888853, 0.04850547, 0.04820495, 0.04541986,
       0.04555095, 0.04527141, 0.04708683, 0.04582071, 0.04404685,
       0.04157262, 0.03785525, 0.03844033, 0.03924443, 0.03806671,
       0.03880695, 0.04006411, 0.04253744, 0.04199351, 0.04262579,
       0.04021978, 0.04440309, 0.04765095, 0.04754462, 0.04790079,
       0.04830547, 0.04706538, 0.04816124, 0.048605  , 0.04583666,
       0.04491841, 0.04453504, 0.04351856, 0.04228246, 0.04237744,
       0.04242139, 0.04030139, 0.04239414, 0.04135653, 0.04020531,
       0.0391979 , 0.03481933, 0.03745275, 0.03729382, 0.0391593 ,
       0.04070108, 0.04356644, 0.04235908, 0.04321472, 0.04296319,
       0.04262355, 0.04263261, 0.04309738, 0.04297469, 0.04161

In [42]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(14, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


,2009-11-30,2009-10-31,2009-09-30,2009-08-31,2009-07-31,2009-06-30,2009-05-31,2009-04-30,2009-03-31,2009-02-28,...,1996-06-30,1996-05-31,1996-04-30,1996-03-31,1996-02-29,1996-01-31,1995-12-31,1995-11-30,1995-10-31,1995-09-30
0,1.00,1.04,1.20,1.37,1.65,2.08,2.56,3.12,4.42,4.39,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.15,1.32,1.49,1.63,1.98,2.36,2.78,3.41,4.63,4.62,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.33,1.58,1.76,1.91,2.27,2.60,2.95,3.57,4.65,4.64,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,1.51,1.79,1.97,2.12,2.49,2.81,3.14,3.76,4.74,4.80,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,1.78,2.09,2.26,2.40,2.77,3.11,3.43,4.05,4.95,5.04,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,2.08,2.43,2.57,2.73,3.08,3.46,3.75,4.35,5.20,5.30,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,2.39,2.77,2.87,3.03,3.38,3.79,4.08,4.60,5.38,5.49,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,2.70,3.06,3.16,3.31,3.66,4.10,4.41,4.81,5.50,5.62,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,3.02,3.34,3.46,3.59,3.93,4.39,4.75,5.03,5.63,5.76,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,3.33,3.61,3.75,3.87,4.20,4.66,5.08,5.27,5.81,5.95,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [43]:
Raw_FTSE_YC_Tot = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot

,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,1996-06-30 00:00:00,1996-05-31 00:00:00,1996-04-30 00:00:00,1996-03-31 00:00:00,1996-02-29 00:00:00,1996-01-31 00:00:00,1995-12-31 00:00:00,1995-11-30 00:00:00,1995-10-31 00:00:00,1995-09-30 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [44]:
raw_liab_ibt_dict_tot={}
tr = Raw_FTSE_YC_Tot['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot[col])

952

In [45]:
int_df_tot = pd.DataFrame(raw_liab_ibt_dict_tot)
int_df_tot = int_df_tot.drop(['Date'], axis=1)
int_df_tot = int_df_tot.iloc[:, ::-1]
int_df_tot

,1995-09-30,1995-10-31,1995-11-30,1995-12-31,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
1,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
2,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
3,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
4,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
948,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
949,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
950,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599


In [46]:
liab_plan_dict_totdf={}
for col in int_df_tot.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot[col][j]/100)**DF[j]))
    liab_plan_dict_totdf[col] = temp_pv
liab_plan_dict_totdf

{Timestamp('1995-09-30 00:00:00'): 7865692887.397379,
 Timestamp('1995-10-31 00:00:00'): 8005377928.816404,
 Timestamp('1995-11-30 00:00:00'): 8207866009.027996,
 Timestamp('1995-12-31 00:00:00'): 8447930221.932001,
 Timestamp('1996-01-31 00:00:00'): 8367212177.477848,
 Timestamp('1996-02-29 00:00:00'): 7826721252.465281,
 Timestamp('1996-03-31 00:00:00'): 7622498455.844169,
 Timestamp('1996-04-30 00:00:00'): 7373621409.616289,
 Timestamp('1996-05-31 00:00:00'): 7340139796.042628,
 Timestamp('1996-06-30 00:00:00'): 7453231672.654455,
 Timestamp('1996-07-31 00:00:00'): 7398936839.97344,
 Timestamp('1996-08-31 00:00:00'): 7252136052.81826,
 Timestamp('1996-09-30 00:00:00'): 7456080191.4033785,
 Timestamp('1996-10-31 00:00:00'): 7773096820.606378,
 Timestamp('1996-11-30 00:00:00'): 7986834103.891992,
 Timestamp('1996-12-31 00:00:00'): 7715816501.556407,
 Timestamp('1997-01-31 00:00:00'): 7607799308.775687,
 Timestamp('1997-02-28 00:00:00'): 7586885912.365833,
 Timestamp('1997-03-31 00:00:

In [47]:
pv_df_totdf= pd.DataFrame(liab_plan_dict_totdf, index = ['Present Values']).transpose()
np.negative(pv_df_totdf['Present Values'])
pv_df_totdf.tail

<bound method NDFrame.tail of             Present Values
1995-09-30    7.865693e+09
1995-10-31    8.005378e+09
1995-11-30    8.207866e+09
1995-12-31    8.447930e+09
1996-01-31    8.367212e+09
...                    ...
2021-05-31    1.492173e+10
2021-06-30    1.556223e+10
2021-07-31    1.594887e+10
2021-08-31    1.580312e+10
2021-09-30    1.544607e+10

[313 rows x 1 columns]>

In [48]:
Plan_Return_YC_Totdf = np.zeros(len(liab_plan_dict_totdf)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf[i] = ((pv_df_totdf['Present Values'][i+1])/pv_df_totdf['Present Values'][i])-1
Plan_Returns_YC_Totdf = pd.DataFrame(Plan_Return_YC_Totdf)
Plan_Returns_YC_Totdf

,0
0,0.017759
1,0.025294
2,0.029248
3,-0.009555
4,-0.064596
...,...
307,0.002709
308,0.042924
309,0.024844
310,-0.009138


In [49]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf.to_excel(writer, sheet_name='returns')
pv_df_totdf.to_excel(writer, sheet_name='pv')
writer.save()

Test Liability Return Data Pre-2010

In [50]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 130)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,2011-10-31 00:00:00,2011-09-30 00:00:00,2011-08-31 00:00:00,2011-07-31 00:00:00,2011-06-30 00:00:00,2011-05-31 00:00:00,2011-04-30 00:00:00,2011-03-31 00:00:00,2011-02-28 00:00:00,2011-01-31 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,0.978098,1.200821,1.016695,0.834075,0.740336,0.691775,0.651798,0.684030,0.800783,0.855293
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,1.058504,1.289367,1.102144,0.904379,0.884920,0.798170,0.811379,0.933030,0.939382,0.966797
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,1.104752,1.336674,1.144379,0.926479,1.013526,0.896584,0.959028,1.175297,1.109245,1.082385
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,1.160063,1.394062,1.164494,0.970688,1.138191,1.032233,1.135948,1.380596,1.287303,1.229226
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,1.357872,1.592194,1.335450,1.155110,1.392998,1.285085,1.422351,1.695042,1.587769,1.486277
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,1.577038,1.808680,1.549877,1.382992,1.690475,1.562437,1.735382,2.049367,1.934109,1.780979
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,1.822674,2.052670,1.797824,1.664563,2.010606,1.865400,2.058998,2.393178,2.266879,2.097326
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,2.094877,2.323251,2.071329,1.985906,2.341357,2.188019,2.384139,2.711216,2.571832,2.426308
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,2.389703,2.605332,2.371506,2.317872,2.684802,2.517224,2.709798,3.024671,2.879268,2.760887
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,2.694074,2.882730,2.687323,2.642271,3.030853,2.837831,3.028868,3.337587,3.193259,3.091951


In [51]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(1, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


,2010-12-31,2010-11-30,2010-10-31,2010-09-30,2010-08-31,2010-07-31,2010-06-30,2010-05-31,2010-04-30,2010-03-31,...,1996-06-30,1996-05-31,1996-04-30,1996-03-31,1996-02-29,1996-01-31,1995-12-31,1995-11-30,1995-10-31,1995-09-30
0,0.88,0.90,0.83,0.90,0.98,1.04,1.34,1.32,0.80,0.83,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.04,0.99,0.90,1.01,1.09,1.16,1.50,1.53,1.07,1.18,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.13,1.06,0.94,1.06,1.18,1.25,1.59,1.70,1.33,1.46,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,1.27,1.13,0.97,1.13,1.25,1.35,1.70,1.86,1.54,1.69,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,1.54,1.32,1.12,1.29,1.40,1.52,1.89,2.10,1.83,1.99,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,1.87,1.56,1.31,1.48,1.60,1.73,2.15,2.40,2.17,2.34,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,2.22,1.83,1.54,1.70,1.82,1.97,2.43,2.68,2.50,2.69,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,2.56,2.11,1.81,1.93,2.05,2.23,2.69,2.93,2.79,3.02,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,2.90,2.39,2.06,2.16,2.26,2.48,2.90,3.15,3.06,3.31,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,3.22,2.67,2.31,2.39,2.48,2.73,3.09,3.36,3.33,3.57,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [52]:
Raw_FTSE_YC_Tot_1 = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot_1

,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,1996-06-30 00:00:00,1996-05-31 00:00:00,1996-04-30 00:00:00,1996-03-31 00:00:00,1996-02-29 00:00:00,1996-01-31 00:00:00,1995-12-31 00:00:00,1995-11-30 00:00:00,1995-10-31 00:00:00,1995-09-30 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [53]:
raw_liab_ibt_dict_tot_1={}
tr = Raw_FTSE_YC_Tot_1['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot_1[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot_1.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot_1[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot_1[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot_1[col])

952

In [54]:
int_df_tot_1 = pd.DataFrame(raw_liab_ibt_dict_tot_1)
int_df_tot_1 = int_df_tot_1.drop(['Date'], axis=1)
int_df_tot_1 = int_df_tot_1.iloc[:, ::-1]
int_df_tot_1[col]

0      5.98
1      5.98
2      5.98
3      5.98
4      5.98
       ... 
947    7.34
948    7.34
949    7.34
950    7.34
951    7.34
Name: 1995-09-30 00:00:00, Length: 952, dtype: float64

In [55]:
liab_plan_dict_totdf_1={}
for col in int_df_tot_1.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot_1[col][j]/100)**DF[j]))
    liab_plan_dict_totdf_1[col] = temp_pv
liab_plan_dict_totdf_1

{Timestamp('1995-09-30 00:00:00'): 7865692887.397379,
 Timestamp('1995-10-31 00:00:00'): 8005377928.816404,
 Timestamp('1995-11-30 00:00:00'): 8207866009.027996,
 Timestamp('1995-12-31 00:00:00'): 8447930221.932001,
 Timestamp('1996-01-31 00:00:00'): 8367212177.477848,
 Timestamp('1996-02-29 00:00:00'): 7826721252.465281,
 Timestamp('1996-03-31 00:00:00'): 7622498455.844169,
 Timestamp('1996-04-30 00:00:00'): 7373621409.616289,
 Timestamp('1996-05-31 00:00:00'): 7340139796.042628,
 Timestamp('1996-06-30 00:00:00'): 7453231672.654455,
 Timestamp('1996-07-31 00:00:00'): 7398936839.97344,
 Timestamp('1996-08-31 00:00:00'): 7252136052.81826,
 Timestamp('1996-09-30 00:00:00'): 7456080191.4033785,
 Timestamp('1996-10-31 00:00:00'): 7773096820.606378,
 Timestamp('1996-11-30 00:00:00'): 7986834103.891992,
 Timestamp('1996-12-31 00:00:00'): 7715816501.556407,
 Timestamp('1997-01-31 00:00:00'): 7607799308.775687,
 Timestamp('1997-02-28 00:00:00'): 7586885912.365833,
 Timestamp('1997-03-31 00:00:

In [56]:
pv_df_totdf_1= pd.DataFrame(liab_plan_dict_totdf_1, index = ['Present Values']).transpose()
np.negative(pv_df_totdf_1['Present Values'])
pv_df_totdf_1

,Present Values
1995-09-30,7.865693e+09
1995-10-31,8.005378e+09
1995-11-30,8.207866e+09
1995-12-31,8.447930e+09
1996-01-31,8.367212e+09
...,...
2021-05-31,1.492173e+10
2021-06-30,1.556223e+10
2021-07-31,1.594887e+10
2021-08-31,1.580312e+10


In [57]:
Plan_Return_YC_Totdf_1 = np.zeros(len(liab_plan_dict_totdf_1)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf_1)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf_1[i] = ((pv_df_totdf_1['Present Values'][i+1])/pv_df_totdf_1['Present Values'][i])-1
Plan_Returns_YC_Totdf_1 = pd.DataFrame(Plan_Return_YC_Totdf_1)
Plan_Returns_YC_Totdf_1

,0
0,0.017759
1,0.025294
2,0.029248
3,-0.009555
4,-0.064596
...,...
307,0.002709
308,0.042924
309,0.024844
310,-0.009138


In [58]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010_1.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf_1.to_excel(writer, sheet_name='returns')
pv_df_totdf_1.to_excel(writer, sheet_name='pv')
writer.save()

In [59]:
Plan_PV_DR_ERF = np.zeros(len(df_t))
for j in range (len(df_t)):
    for i in range (j,len(Time_CF_Tot)):
        Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
Plan_PV_DR_ERF[240]

10725287202.018148

In [60]:
#fullfillment_return = .045
#MV_Assets = 13203547000


In [61]:
#Plan_Asset_MV = np.zeros(len(df_t))
#for i in range(len(df_t)):
       #if (i == 0):
            
                #Plan_Asset_MV[i] = MV_Assets
        #else:
            
                #Plan_Asset_MV[i] = (Plan_Asset_MV[i-1]*(1+fullfillment_return)**DF[0].tolist())-CF[i-1]
#Plan_Asset_MV

In [62]:
#df_Asset_MV = pd.DataFrame(Plan_Asset_MV)
#df_Asset_MV

In [63]:
#FR = Plan_Asset_MV/Plan_PV_DR_ERF
#FR = pd.DataFrame(FR)
#FR.style.format("{:.2%}")


In [64]:
#FS = Plan_Asset_MV[-1] - Plan_PV_DR_ERF[-1] 
#FS

In [65]:
def full_solve(x0):
    return np.asscalar(fsolve(fullfillment_solve, x0=x0))


def fullfillment_solve(fullfillment_return):
    #fullfillment_return = .045
    MV_Assets = 13203547000
    Plan_PV_DR_ERF = np.zeros(len(df_t))
    Plan_Asset_MV = np.zeros(len(df_t))
    x = Years_to_FF/DF[0]
    x = x.astype(int)
    
    for j in range(len(df_t)):
        if (j == 0):
            for i in range(j,len(Time_CF_Tot)):
                Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = MV_Assets
    
        else:
            for i in range(j,len(Time_CF_Tot)):
                Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF[j-1]
    
    return Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]


    

In [66]:
fullfill_irr = full_solve(x0=.01)
print("Base Case Fullfillment rate of return:%3.8f"%fullfill_irr)


Base Case Fullfillment rate of return:0.03840666


In [67]:
fullfillment_return = fullfill_irr
MV_Assets = 13203547000
Plan_PV_DR_ERF = np.zeros(len(df_t))
Plan_Asset_MV = np.zeros(len(df_t))
x = Years_to_FF/DF[0]
x = x.astype(int)
for j in range(len(df_t)):
    if (j == 0):
        for i in range(j,len(Time_CF_Tot)):
            Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = MV_Assets
    
    else:
        for i in range(j,len(Time_CF_Tot)):
            Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF[j-1]
    
Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]

2.288818359375e-05

In [68]:
Excess_Return = fullfill_irr - Plan_DR['IRR'][-1]
#Excess_Return = pd.DataFrame(Excess_Return)
#Excess_Return.style.format("{:.2%}")
Excess_Return

0.006006658632797826